In [1]:
import os
import shutil
import subprocess
import time
import configparser
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from PyQt5 import QtWidgets, uic
from PyQt5.QtWidgets import QMessageBox, QFileDialog

class YourClassName:
    def extract_and_save_not_filed(self):
        # ... (your existing code for Excel operations)
        if not hasattr(self, 'file_1') or not hasattr(self, 'file_2'):
            QMessageBox.warning(self, 'Warning', 'Please select both File 1 and File 2')
            return

        start_time = time.time()
        data_file_1 = pd.read_excel('client data.xlsx')
        data_file_2 = pd.read_excel('GST2A.xlsx')

        matched_gst = []
        for gst_no_1 in data_file_1['GSTIN/UIN']:
            matched_gst.extend(
                gst_no_1
                for gst_no_2 in data_file_2['GSTIN/UIN']
                if gst_no_1 == gst_no_2
            )

        matched_rows = pd.merge(data_file_1, data_file_2, on='GSTIN/UIN', how='inner')

        wrong_matched_gst = []
        counter = []
        client = []
        count = 0  # Initialize count variable
        for gst_no_1 in data_file_1['GSTIN/UIN']:
            count += 1
            for gst_no_2 in data_file_2['GSTIN/UIN']:
                if not pd.isna(gst_no_1) and not pd.isna(gst_no_2) and gst_no_1 != gst_no_2 and gst_no_1[2:12] == gst_no_2[2:12]:
                    wrong_matched_gst.extend([gst_no_1])
                    client.extend([gst_no_2])
                    counter.extend([count])

        result_df = pd.DataFrame({'Index in client sheet': counter,'Correct GST No.': client, 'Wrong Matched GST': wrong_matched_gst})
        
        output_file_wrong_match = self.generate_excel_sheet('Matched But Not Fully', self.header_content, result_df)
        output_file_matched_rows = self.generate_excel_sheet('Matched Rows', self.header_content, matched_rows)

        self.pushButton_4.setVisible(True)
        self.result_sheet_path_wrong_match = output_file_wrong_match
        self.result_sheet_path_matched_rows = output_file_matched_rows

class LoginDialog(QtWidgets.QDialog):
    def __init__(self):
        super(LoginDialog, self).__init__()
        uic.loadUi("login.ui", self)

        # Connect the login button
        self.pushButton.clicked.connect(self.login)
        # Connect the convert button
        self.pushButton_4.clicked.connect(self.convert_and_download)

        self.file_1 = None
        self.file_2 = None

    def login(self):
        # ... (your existing login code)
        config = configparser.ConfigParser()
        config.read("config.ini")

        entered_username = self.lineEdit.text()
        entered_password = self.lineEdit_2.text()

        if entered_username in config['Users']:
            stored_password_plain = config['Users'][entered_username]

            if stored_password_plain == entered_password:
                self.accept()  # Close the login window
                convert_dialog = ConvertDialog()
                if convert_dialog.exec_() == QtWidgets.QDialog.Accepted:
                    pass  # Conversion is complete
            else:
                QMessageBox.warning(self, 'Login Failed', 'Incorrect username or password')
        else:
            QMessageBox.warning(self, 'Login Failed', 'Incorrect username or password')

    def convert_and_download(self):
        if self.file_1 is None or self.file_2 is None:
            QMessageBox.warning(self, 'File Selection Error', 'Please select both files before converting.')
            return

        # Instantiate YourClassName
        excel_processor = YourClassName()

        # Perform Excel operations
        excel_processor.extract_and_save_not_filed()

        # Get the result DataFrames
        not_fully_matched_df = excel_processor.result_df
        matched_rows_df = excel_processor.matched_rows

        # Define header
        header = {'Prepare By': 'ABC .(Chartered Accountants) Jaipur',
                  'Client Name': 'XYZ LIMITED (08AAFCA9416Q1Z2)',
                  'Client GSTIN No': '08AAFCA9416Q1Z2',
                  'Period': '2022-2023',
                  'Date of Report': '31-Aug-23',
                  'Subject': 'GST 2A Reconciliation with Books Input tax Credit',
                  'Sub-Subject': 'This sheet contains Invoices which are matched with the GST-2A'}

        # Generate Excel sheets
        not_fully_matched_path = excel_processor.generate_excel_sheet('NotFullyMatched', header, not_fully_matched_df, [])
        matched_rows_path = excel_processor.generate_excel_sheet('Matched', header, matched_rows_df, [])

        # Ask the user to choose a location for saving the files
        save_location = QFileDialog.getExistingDirectory(self, 'Select Save Location')

        # Copy the files to the chosen location
        shutil.copy(not_fully_matched_path, os.path.join(save_location, 'NotFullyMatched.xlsx'))
        shutil.copy(matched_rows_path, os.path.join(save_location, 'Matched.xlsx'))

        # Show a success message
        QMessageBox.information(self, 'Download Complete', 'Files downloaded successfully!')

        # Optionally, open the folder
        subprocess.Popen(['explorer', save_location])

    def browseFile1(self):
        file_1, _ = QFileDialog.getOpenFileName(None, 'Select File 1', '', 'Excel Files (*.xlsx);;All Files (*)')
        if file_1:
            self.file_1 = file_1
            self.pushButton_3.setText(file_1)  # Set the selected file name as text

    def browseFile2(self):
        file_2, _ = QFileDialog.getOpenFileName(None, 'Select File 2', '', 'Excel Files (*.xlsx);;All Files (*)')
        if file_2:
            self.file_2 = file_2
            self.pushButton_2.setText(file_2)  # Set the selected file name as text

if __name__ == "__main__":
    app = QtWidgets.QApplication([])
    login_dialog = LoginDialog()
    login_dialog.show()
    app.exec_()


AttributeError: 'LoginDialog' object has no attribute 'pushButton_4'

In [ ]:
import sys
import os
import time
from PyQt5 import QtWidgets, uic
from PyQt5.QtWidgets import QFileDialog, QMessageBox
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import configparser


class ConvertDialog(QtWidgets.QDialog):
    def __init__(self, parent=None):
        super(ConvertDialog, self).__init__(parent)
        uic.loadUi("C:/Users/infiq/OneDrive/Desktop/GSTR-2A soft. Exp/Data Analysis-20231109T082735Z-001/Data Analysis/convert.ui", self)

        # Connect buttons and signals
        self.pushButton_3.clicked.connect(self.browseFile1)
        self.pushButton_2.clicked.connect(self.browseFile2)
        self.pushButton.clicked.connect(self.extract_and_save_not_filed)
        self.pushButton_4.clicked.connect(self.downloadResult)

        # Hide the "Download" button initially
        self.pushButton_4.setVisible(False)

        # Additional code for header_content
        self.header_content = {'Prepare By': 'ABC .(Chartered Accountants) Jaipur',
                               'Client Name': 'XYZ LIMITED (08AAFCA9416Q1Z2)',
                               'Clint GSTIN No': '08AAFCA9416Q1Z2',
                               'Period': '2022-2023',
                               'Date of Report': '31-Aug-23',
                               'Subject': 'GST 2A Reconciliation with Books Input tax Credit',
                               'Sub-Subject': 'This sheet contains Invoices which are matched with the GST-2A'}

    def browseFile1(self):
        file_1, _ = QFileDialog.getOpenFileName(None, 'Select File 1', '', 'Excel Files (*.xlsx);;All Files (*)')
        if file_1:
            self.file_1 = file_1
            self.pushButton_3.setText(file_1)  # Set the selected file name as text

    def browseFile2(self):
        file_2, _ = QFileDialog.getOpenFileName(None, 'Select File 2', '', 'Excel Files (*.xlsx);;All Files (*)')
        if file_2:
            self.file_2 = file_2
            self.pushButton_2.setText(file_2)  # Set the selected file name as text

    def generate_excel_sheet(self, sheet_name, header_content, data):
        # Create a workbook and add a sheet
        workbook = Workbook()
        current_sheet = workbook.active
        current_sheet.title = sheet_name

        # Call the function to add the header
        self.add_header(current_sheet, header_content)

        # Convert the DataFrame to rows and append it to the sheet from row 8
        for row in dataframe_to_rows(data, index=False, header=True):
            current_sheet.append(row)

        # Save the workbook
        output_file_path = f"{sheet_name}.xlsx"
        workbook.save(output_file_path)

        return output_file_path

    def add_header(self, sheet, header_content):
        for idx, (key, value) in enumerate(header_content.items(), start=1):
            sheet.cell(row=idx, column=1).value = key
            sheet.cell(row=idx, column=2).value = value

    def extract_and_save_not_filed(self):
        if not hasattr(self, 'file_1') or not hasattr(self, 'file_2'):
            QMessageBox.warning(self, 'Warning', 'Please select both File 1 and File 2')
            return

        start_time = time.time()
        data_file_1 = pd.read_excel(self.file_1)
        data_file_2 = pd.read_excel(self.file_2)

        matched_gst = []
        for gst_no_1 in data_file_1['GSTIN/UIN']:
            matched_gst.extend(
                gst_no_1
                for gst_no_2 in data_file_2['GSTIN/UIN']
                if gst_no_1 == gst_no_2
            )

        not_filed_data = data_file_2[data_file_2['3B Status'] == 'Not Filed']
        not_filed_data = not_filed_data[not_filed_data['GSTIN/UIN'].isin(matched_gst)]

        # I try to make non matched gst fully
        wrong_matched_gst = []
        for gst_no_1 in data_file_1['GSTIN/UIN']:
            wrong_matched_gst.extend(
                gst_no_1
                for gst_no_2 in data_file_2['GSTIN/UIN']
                if gst_no_1 == gst_no_2:
                    if gst_no_1[2:len(gst_no_1)-3] == gst_no_2[2:len(gst_no_2)-3]
            )

        output_file_wrong_match = self.generate_excel_sheet('Matched But Not Fully', self.header_content, wrong_matched_gst)

        output_file = self.generate_excel_sheet('3B StatusNotFilled', self.header_content, not_filed_data)
        self.pushButton_4.setVisible(True)  # Show the "Download" button

        # Store the paths for later use
        self.result_sheet_path = output_file
        self.wrong_match_sheet_path = output_file_wrong_match

        end_time = time.time()
        total_time = round(end_time - start_time, 2)
        QMessageBox.information(self, 'Processing Complete', f'Congratulations! Your data has been processed in {total_time} seconds.')

    def downloadResult(self):
        if hasattr(self, 'result_sheet_path'):
            download_location, _ = QFileDialog.getSaveFileName(
                None, 'Save Result Sheet', '', 'Excel Files (*.xlsx);;All Files (*)'
            )
            if download_location:
                with open(download_location, 'wb') as f:
                    f.write(open(self.result_sheet_path, 'rb').read())
                os.startfile(download_location)  # Opens the file with the default application

        if hasattr(self, 'wrong_match_sheet_path'):
            download_location_wrong, _ = QFileDialog.getSaveFileName(
                None, 'Save Wrong Match Sheet', '', 'Excel Files (*.xlsx);;All Files (*)'
            )
            if download_location_wrong:
                with open(download_location_wrong, 'wb') as f:
                    f.write(open(self.wrong_match_sheet_path, 'rb').read())
                os.startfile(download_location_wrong)  # Opens the file with the default application


class LoginDialog(QtWidgets.QDialog):
    def __init__(self):
        super(LoginDialog, self).__init__()
        uic.loadUi("login.ui", self)

        # Connect the login button
        self.pushButton.clicked.connect(self.login)

    def login(self):
        # Load the configuration file
        config = configparser.ConfigParser()
        config.read("config.ini")

        entered_username = self.lineEdit.text()
        entered_password = self.lineEdit_2.text()

        if entered_username in config['Users']:
            stored_password_plain = config['Users'][entered_username]

            if stored_password_plain == entered_password:
                self.accept()  # Close the login window
                convert_dialog = ConvertDialog(self)
                if convert_dialog.exec_() == QtWidgets.QDialog.Accepted:
                    pass  # Conversion is complete
            else:
                QMessageBox.warning(self, 'Login Failed', 'Incorrect username or password')
        else:
            QMessageBox.warning(self, 'Login Failed', 'Incorrect username or password')


def main():
    app = QtWidgets.QApplication(sys.argv)
    login_dialog = LoginDialog()
    if login_dialog.exec_() == QtWidgets.QDialog.Accepted:
        sys.exit(app.exec_())


if __name__ == "__main__":
    main()
